<a href="https://colab.research.google.com/github/kaditi14/LP-I-/blob/main/final_HPC_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile hpc3.cpp

#include<iostream>
#include<omp.h>
using namespace std;

#define MERGE_SORT_PARALLEL 0
#define MERGE_SORT_SERIAL 1
#define BUBBLE_SORT_PARALLEL 2
#define BUBBLE_SORT_SERIAL 3

template<class T>
class Array {
    
  public:
  int size;
  T* arr;

  //Parameterized Constructer
  Array(int n, bool init_rand=true){
    size = n;
    arr = (T*)malloc(n * sizeof(T));
    if(init_rand==true){
      initialize_random();
    }
  }

  //Copy Constructor
  Array(const Array &arg){
      size = arg.size;
      arr = (T*)malloc(size * sizeof(T));
      for(int i=0;i<size;i++){
          arr[i] = arg.arr[i];
      }
  }

  void initialize_random() {
    for(int i=0; i<size; i++) {
      arr[i] = rand()%100 * 1.43;
    }
  }

  //Overloading Insertion Operator
  template <typename U>   
  friend ostream& operator<<(ostream& os, const Array<U> &obj);


  void merge(int l, int mid, int h, T* temp) {
      
    int cur = l;
    int i = l;
    int j = mid+1;

    while(i <= mid && j <= h){
      if(arr[i] < arr[j]) {
        temp[cur] = arr[i];
        cur++;
        i++;
      }
      else {
        temp[cur] = arr[j];
        cur++;
        j++;
      }
    }

    if(i <= mid) {
      while(i <= mid) {
        temp[cur] = arr[i];
        i++;
        cur++;
      }
    }
    else if(j <= h) {
      while(j <= h) {
        temp[cur] = arr[j];
        j++;
        cur++;
      }
    }

    for(int arrIndex=l;arrIndex<=h;arrIndex++){
        arr[arrIndex] = temp[arrIndex];
    }

  } // end of merge()

  void mergeSortParallel(int l, int h, T* tmp)
  {
    if (h <= l)
      return;
   
    int mid = l + (h-l)/2;
    int mid2 = mid+1;

    #pragma omp task firstprivate (l, mid, tmp)
    mergeSortParallel(l, mid, tmp);

    #pragma omp task firstprivate (mid2, h, tmp)
    mergeSortParallel(mid2, h, tmp);

    #pragma omp taskwait
    merge(l,mid, h, tmp);
   
  }


  void mergeSortParallel(){
    T* tmp = (T*)malloc(size * sizeof(T));
    omp_set_num_threads(2);
    #pragma omp parallel
    {
      #pragma omp single
      mergeSortParallel(0, size-1, tmp);
    } 
  }

  void mergeSortSerial(int l, int h, T* tmp)
  {
    if (h <= l)
      return;
    int mid = l + (h-l)/2;
    mergeSortSerial(l, mid, tmp);
    mergeSortSerial(mid+1, h, tmp);
    merge(l,mid, h, tmp);
   
  }


  void mergeSortSerial(){
    T* tmp = (T*)malloc(size * sizeof(T));
    mergeSortSerial(0, size-1, tmp);
  }

  void swap(T *num1, T *num2) {
      T temp = *num1;
      *num1 = *num2;
      *num2 = temp;
  }

  void bubbleSortParallel(){
      for(int i=size-1; i>=0; i--) {
          #pragma omp parallel for
          for(int j=0; j<i; j++) {
              if(arr[j] > arr[j+1])
                swap(&arr[j], &arr[j+1]);
          }
      }
      
  }

  
  void bubbleSortSerial(){
      for(int i=size-1; i>=0; i--) {
          for(int j=0; j<i; j++) {
              if(arr[j]>arr[j+1]){
                  swap(&arr[j],&arr[j+1]);
              }
          }
      }
  }
  

  double timeit(int choice){
      
      double start, end;

      switch(choice){
          case MERGE_SORT_PARALLEL:
              start = omp_get_wtime();
              mergeSortParallel();
              end = omp_get_wtime();
              break;
          case MERGE_SORT_SERIAL:
              start = omp_get_wtime();
              mergeSortSerial();
              end = omp_get_wtime();
              break;
          case BUBBLE_SORT_PARALLEL:
              start = omp_get_wtime();
              bubbleSortParallel();
              end = omp_get_wtime();
              break;
          case BUBBLE_SORT_SERIAL:
              start = omp_get_wtime();
              bubbleSortSerial();
              end = omp_get_wtime();
              break;
          default:
              return 0;

      }

      return (end-start);

  }

};


int timeAlgorithms(int n){

    Array <float> obj(n), a=obj, b=obj, c=obj, d=obj, e=obj, f=obj;

    cout<<"\nNUMBER_OF_ELEMENTS = "<<n;

    cout<<"\nMERGE_SORT_SERIAL Time = "     <<a.timeit(MERGE_SORT_SERIAL);
    cout<<"\nMERGE_SORT_PARALLEL Time = "   <<b.timeit(MERGE_SORT_PARALLEL);
    cout<<"\nBUBBLE_SORT_SERIAL Time = "    <<c.timeit(BUBBLE_SORT_SERIAL);
    cout<<"\nBUBBLE_SORT_PARALLEL Time = "  <<d.timeit(BUBBLE_SORT_PARALLEL);
  }

template <typename T>
ostream& operator<<(ostream& os, const Array<T> &obj){
  if(obj.size==0){
    os<<"[ ]";
    return os;
  }
  os<<"[ "<<obj.arr[0];
  for(int i=1; i<obj.size; i++) {
    os<<", "<<obj.arr[i];
  }
  os<<"]";
  return os;
}

int main(){
    
    Array <float> obj(8), a=obj, b=obj, c=obj, d=obj;

    cout << "\n\nORIGINAL ARRAY : \n"<< obj;

    a.mergeSortParallel();
    cout << "\n\nMERGE_SORT_PARALLEL : \n"<< a;

    b.mergeSortSerial();
    cout << "\n\nMERGE_SORT_SERIAL : \n"<< b;

    c.bubbleSortParallel();
    cout << "\n\nBUBBLE_SORT_PARALLEL : \n"<< c;

    d.bubbleSortSerial();
    cout << "\n\nBUBBLE_SORT_SERIAL : \n"<< d;

    cout<<endl<<endl;
    timeAlgorithms(65536);

    return 0;
}

Overwriting hpc3.cpp


In [ ]:
!g++ -fopenmp hpc3.cpp
!./a.out



ORIGINAL ARRAY : 
[ 118.69, 122.98, 110.11, 21.45, 132.99, 50.05, 122.98, 131.56]

MERGE_SORT_PARALLEL : 
[ 21.45, 50.05, 110.11, 118.69, 122.98, 122.98, 131.56, 132.99]

MERGE_SORT_SERIAL : 
[ 21.45, 50.05, 110.11, 118.69, 122.98, 122.98, 131.56, 132.99]

BUBBLE_SORT_PARALLEL : 
[ 21.45, 50.05, 110.11, 118.69, 122.98, 122.98, 131.56, 132.99]

BUBBLE_SORT_SERIAL : 
[ 21.45, 50.05, 110.11, 118.69, 122.98, 122.98, 131.56, 132.99]


NUMBER_OF_ELEMENTS = 65536
MERGE_SORT_SERIAL Time = 0.0121885
MERGE_SORT_PARALLEL Time = 0.0327736
BUBBLE_SORT_SERIAL Time = 16.9729
BUBBLE_SORT_PARALLEL Time = 13.4579